# Orbital element conversion with GRSS between cometary and cartesian coordinates

In [1]:
import numpy as np
np.set_printoptions(linewidth=100)

from grss import prop, fit, utils

In [2]:
# Specify body and get orbit from SBDB
body_id = '99942'
orbit, cov, nongrav_info = fit.get_sbdb_info(body_id)

In [3]:
# GRSS uses MJD times and radian angles for orbit input
t0 = orbit['t']
e = orbit['e']
q = orbit['q']
tp = orbit['tp']
om = orbit['om']
w = orbit['w']
i = orbit['i']
com_elems = np.array([e, q, tp, om, w, i])
print(f'Time: {t0} MJD TDB')
print(f'Heliocentric ecliptic cometary elements: {com_elems}')

Time: 59215.0 MJD TDB
Heliocentric ecliptic cometary elements: [1.91521689e-01 7.45827048e-01 5.91005394e+04 3.56115108e+00 2.21049531e+00 5.82372968e-02]


In [4]:
# com_elems are heliocentric ecliptic elements, so cart_elems are heliocentric ecliptic cartesian elements
cart_elems_eclip = np.array(prop.cometary_to_cartesian(t0, com_elems))
print(f'Heliocentric ecliptic cartesian elements: {cart_elems_eclip}')

Heliocentric ecliptic cartesian elements: [-4.09853084e-01  9.62164847e-01 -6.09660414e-02 -1.50755241e-02 -4.10795545e-03 -1.39312966e-04]


In [5]:
# we can rotate the ecliptic cartesian elements to equatorial cartesian elements
state_rot = np.zeros((6, 6))
state_rot[:3, :3] = prop.eclip2equat
state_rot[3:, 3:] = prop.eclip2equat
cart_elems_equat = state_rot@cart_elems_eclip
print(f'Heliocentric equatorial cartesian elements: {cart_elems_equat}')

Heliocentric equatorial cartesian elements: [-0.40985308  0.90701989  0.32679195 -0.01507552 -0.00371356 -0.00176187]


In [6]:
# rotate heliocentric ecliptic elements back to heliocentric cometary elements
com_elems_2 = np.array(prop.cartesian_to_cometary(t0, cart_elems_eclip))
print(f'Heliocentric ecliptic cometary elements: {com_elems_2}')

Heliocentric ecliptic cometary elements: [1.91521689e-01 7.45827048e-01 5.91005394e+04 3.56115108e+00 2.21049531e+00 5.82372968e-02]


In [7]:
# create a dummy propagation simulation to query planetary ephemerides
de_kernel = 440
de_kernel_path = utils.default_kernel_path
ephem_sim = prop.PropSimulation("ephemeris sim", t0, de_kernel, de_kernel_path)
# print list of SPICE bodies in sim
spice_bodies = [body.name for body in ephem_sim.spiceBodies]
print(spice_bodies)

['Sun', 'Mercury Barycenter', 'Venus Barycenter', 'Earth', 'Moon', 'Mars Barycenter', 'Jupiter Barycenter', 'Saturn Barycenter', 'Uranus Barycenter', 'Neptune Barycenter', 'Pluto Barycenter', 'Ceres', 'Vesta', 'Pallas', 'Hygiea', 'Davida', 'Interamnia', 'Europa', 'Sylvia', 'Eunomia', 'Juno', 'Psyche', 'Camilla', 'Thisbe', 'Iris', 'Euphrosyne', 'Cybele']


In [8]:
# compute sun state and get barycentric cartesian state of body
ephem_sim.map_ephemeris() # map and unmap ephem before and after using it to avoid memory leaks
sun_state = ephem_sim.get_spiceBody_state(t0, "Sun")
ephem_sim.unmap_ephemeris()
cart_elems_equat_bary = sun_state + cart_elems_equat
print(f'Barycentric equatorial cartesian elements: {cart_elems_equat_bary}')

Barycentric equatorial cartesian elements: [-0.41650502  0.912486    0.32927708 -0.01508236 -0.00371893 -0.00176396]


In [9]:
# -4.165050165607737E-01   9.124859998624349E-01   3.292770803498312E-01  -1.508236398642759E-02  -3.718932684147276E-03  -1.763962050771160E-03
horizons_bary_state = [-4.165050165607737E-01, 9.124859998624349E-01, 3.292770803498312E-01, -1.508236398642759E-02, -3.718932684147276E-03, -1.763962050771160E-03]
diff = cart_elems_equat_bary - horizons_bary_state
pos_error = np.linalg.norm(diff[:3])
vel_error = np.linalg.norm(diff[3:])
au2m = 149597870700
day2s = 86400
print(f'Position error: {pos_error*au2m:0.3f} m')
print(f'Velocity error: {vel_error*au2m/day2s:0.3e} m/s')

Position error: 0.687 m
Velocity error: 1.443e-07 m/s


In [10]:
# -6.651932764979233E-03   5.466113085853516E-03   2.485133157222562E-03  -6.839864192607621E-06  -5.372759267110345E-06  -2.094066270175489E-06
horizons_sun_state = np.array([-6.651932764979233E-03, 5.466113085853516E-03, 2.485133157222562E-03, -6.839864192607621E-06, -5.372759267110345E-06, -2.094066270175489E-06])
diff_sun = sun_state - horizons_sun_state
pos_error_sun = np.linalg.norm(diff_sun[:3])
vel_error_sun = np.linalg.norm(diff_sun[3:])
print(f'Sun position error: {pos_error_sun*au2m:0.3e} m')
print(f'Sun velocity error: {vel_error_sun*au2m/day2s:0.3e} m/s')

Sun position error: 2.595e-07 m
Sun velocity error: 4.038e-14 m/s
